In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter

In [562]:
train_adata = sc.read_csv("C:\\Users\\Lenovo\\Documents\\Python Scripts\\Inter-dataset\\PbmcBench\\CEL-Seq\\CL_pbmc1.csv")
test_adata = sc.read_csv("C:\\Users\\Lenovo\\Documents\\Python Scripts\\Inter-dataset\\PbmcBench\\inDrop\\iD_pbmc1.csv")

In [584]:
train_label = pd.read_csv("C:\\Users\\Lenovo\\Documents\\Python Scripts\\Inter-dataset\\PbmcBench\\CEL-Seq\\CL_pbmc1Labels.csv")
test_label = pd.read_csv("C:\\Users\\Lenovo\\Documents\\Python Scripts\\Inter-dataset\\PbmcBench\\inDrop\\iD_pbmc1Labels.csv")

In [564]:
sc.pp.normalize_total(train_adata, target_sum=1e4)
sc.pp.log1p(train_adata)

sc.pp.normalize_total(test_adata, target_sum=1e4)
sc.pp.log1p(test_adata)

In [565]:
gene = train_adata.var_names & test_adata.var_names

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_48184\783173373.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  gene = train_adata.var_names & test_adata.var_names


In [566]:
data = train_adata.to_df()[gene]
test_data = test_adata.to_df()[gene]

In [567]:
test_data.shape

(21832, 15676)

In [585]:
train_label.columns = ['Label']
train_label.Label.value_counts()
status_dict = train_label['Label'].unique().tolist()
train_label['transfromed']=train_label['Label'].apply(lambda x : status_dict.index(x))
y_train = train_label['transfromed'].values

In [586]:
X_train = data.values
X_test = test_data.values

In [587]:
train_label.Label.value_counts()

Cytotoxic T cell       6799
CD4+ T cell            4472
CD14+ monocyte         3808
B cell                 2362
Megakaryocyte           904
Natural killer cell     850
CD16+ monocyte          559
Name: Label, dtype: int64

In [588]:
status_dict

['CD4+ T cell',
 'Cytotoxic T cell',
 'Natural killer cell',
 'CD16+ monocyte',
 'CD14+ monocyte',
 'Megakaryocyte',
 'B cell']

In [589]:
dtype = torch.float
X_train=torch.from_numpy(X_train)
X_test=torch.from_numpy(X_test)

In [590]:
dtype = torch.float

y_train = torch.from_numpy(y_train)

In [591]:
class_sample_count = torch.tensor(
    [(y_train == t).sum() for t in torch.unique(y_train, sorted=True)])
weight = 1. / class_sample_count.float()
samples_weight = torch.tensor([weight[t] for t in y_train])
sampler = Data.WeightedRandomSampler(samples_weight, len(samples_weight))

In [592]:
train_data= Data.TensorDataset(X_train,y_train)
train_loader= Data.DataLoader(dataset=train_data,batch_size=128,shuffle=True,num_workers=1)

In [593]:
import torch
import torch.nn as nn

class classifier(nn.Module):
    '''

    '''
    def __init__(self, input_size, class_num):
        '''

        :param input_size: Input layer unit number (feature number)
        :param class_num: Number of different cell types.
        '''
        if input_size == None or class_num == None:
            raise ValueError("Must declare number of features and number of classes")
        super(classifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ELU()
        self.dropout1 = nn.Dropout(0.5)
        self.layer2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ELU()
        self.dropout2 = nn.Dropout(0.5)
        self.layer3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.relu3 = nn.ELU()
        self.dropout3 = nn.Dropout(0.5)
        self.layer4 = nn.Linear(32, class_num)


    def forward(self, x):
        '''

        :param x: forward calculation
        :return:
        '''
        out = self.layer1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.layer2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        out = self.layer3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        out = self.layer4(out)

        return out

In [594]:
input_size = X_train.shape[1]
num_class = len(status_dict)
num_epochs = 20

model = classifier(input_size, num_class)

In [595]:
print(model)

classifier(
  (layer1): Linear(in_features=15676, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ELU(alpha=1.0)
  (dropout2): Dropout(p=0.5, inplace=False)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ELU(alpha=1.0)
  (dropout3): Dropout(p=0.5, inplace=False)
  (layer4): Linear(in_features=32, out_features=7, bias=True)
)


In [596]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
total_step = len(train_loader)

In [597]:
import time
print("--------Start annotating----------")
start = time.perf_counter()

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print("Computational unit be used is:", device)

model.to(device)
#loss_weight = loss_weight.to(device)

model.train()
for epoch in range(num_epochs):
    for step,(batch_x,batch_y) in enumerate(train_loader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        train_loss = criterion(outputs, batch_y)
        
        # Backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
    a = "="*(epoch+1)
    b = "."*(num_epochs-epoch-1)
    c = ((epoch+1)/num_epochs)*100
    dur = time.perf_counter() - start
    print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(c,a,b,dur),end = '')
    time.sleep(0.1)
    
print("\n--------Annotation Finished----------")

--------Start annotating----------
Computational unit be used is: cpu
100%[====================->]114.30s
--------Annotation Finished----------


In [598]:
model.eval()
with torch.no_grad():
    X_test = X_test.to(device)
    output = model(X_test)

temp = F.softmax(output, dim=1)
pred = torch.argmax(temp, dim=1).cpu()

result = list(pred.numpy())
results = []
for i in result:
    results.append(status_dict[i])

In [599]:
y_test = test_label

In [600]:
from sklearn.metrics import confusion_matrix,cohen_kappa_score
cohen_kappa_score(y_test,results)

0.9701730671642695

In [421]:
f1_score(y_test,results,average='micro')

0.985358901269759

In [422]:
f1 = f1_score(y_test,results,average=None)
np.median(f1)

0.9824035941594909

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, results)

array([[2432,    2,    0,    1,    1,    0,    0,    0,    0],
       [   0, 3908,    3,    0,    1,    8,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   3,    5,    0, 4474,   28,    0,    0,    0,    0],
       [   1,    2,    0,   24, 6848,    0,    0,   10,    0],
       [   0,    0,    0,    0,    0,  209,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,  904,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,  102]],
      dtype=int64)